# Regression for Used Car Prices in United States


## Team Background

- Project Grp 08

### Team Members

- Team member 1
    - Name: Richard Anton
    - Email: [rna63@drexel.edu](mailto:rna63@drexel.edu)
    
I am a MS CS student at Drexel. I expect to complete my degree in June 2023 at the end of the next quarter.
I have been a software developer since 1995, and have significant experience with Python.

I have some hands on experience with machine learning, and have taken machine learning, deep learning, artificial intellience, computer vision and DSCI-501 courses at Drexel.

I do not have as much background with exploratory data analysis. I do have a fairly solid mathematical background, including basic statistics.

A number of the prior projects that I have worked on have been focused on classification, for example of images or sentiment analysis of textual data.

Since much of my experience with handling data sets for machine learning problems has been through graduate CS courses, I have implemented them typically without a dependency on anything but NumPy and matplotlib for visualizing results, which means I have limited experience with existing common tools like scikit-learn. 

I am competent at using LaTex, but not an advanced user.

I have significant experience with NumPy and some experience using Pandas. I also have experience with matplotlib for visualizing aspects of data, but less with Seaborn.

The areas and skills I would like to grow through this project are:

1. More hands-on experience with scikit-learn and Pandas libraries.
2. More practice with visualization tools, such as matplotlib and Seaborn.
3. More practical experience with investigating a dataset, i.e. what data cleaning is needed and what relationships can be discovered in the data.
4. Experience with a practical machine learning regression problem.
5. Feature selection techniques to make machine learning problems more effective.

In [1]:
# setup imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

import random



In [2]:
sns.set(style="darkgrid")

In [3]:
from IPython.display import display, HTML

In [4]:
class Heading(object):
    template = """<div style="float: left; padding: 10px;">
    <h{size}>{0}</h{size}>
    </div>"""
    def __init__(self, txt, size):
        self.txt = txt
        self.size = size
        
    def _repr_html_(self):
        return self.template.format(self.txt, size=self.size)
    
    def __repr__(self):
        return self.txt

def show_heading(txt, size=2):
    display(Heading(txt, size=size))

In [5]:
#dataset_path = "craigslist_sampled_cleaned_2023_03_05_19_07_36.csv"
dataset_path = "craigslist_full_cleaned_2023_03_12_10_45_22.csv"

target_col = 'price'

In [6]:
# load dataset
orig_df = pd.read_csv(dataset_path)
df = orig_df.copy()
# show a sample for sanity check
df.head()

,price,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,6000.0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,az
1,11900.0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,ar
2,21000.0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,fl
3,1500.0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,ma
4,4900.0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,nc


In [7]:
# split into input data and output values
X_all = df.drop(columns=[target_col])
y_all = df[target_col]

show_heading("X", size=2)
display(X_all)
show_heading("y", size=2)
display(y_all)

X

,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,az
1,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,ar
2,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,fl
3,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,ma
4,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393903,2019.000000,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.000000,clean,other,fwd,full-size,sedan,white,wy
393904,2020.000000,volvo,s60 t5 momentum sedan 4d,good,6 cylinders,gas,12029.000000,clean,other,fwd,full-size,sedan,red,wy
393905,2020.000000,cadillac,xt4 sport suv 4d,good,6 cylinders,diesel,4174.000000,clean,other,4wd,full-size,hatchback,white,wy
393906,2018.000000,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.000000,clean,other,fwd,full-size,sedan,silver,wy


y

0          6000.0
1         11900.0
2         21000.0
3          1500.0
4          4900.0
           ...   
393903    23590.0
393904    30590.0
393905    34990.0
393906    28990.0
393907    30590.0
Name: price, Length: 393908, dtype: float64

In [8]:
# import preprocessing from sklearn
from sklearn import preprocessing

In [9]:
# determine types of columns
numeric_cols = X_all.select_dtypes(include=['int', 'float']).columns
cat_cols = X_all.select_dtypes(include=['object']).columns
bool_cols = X_all.select_dtypes(include=['bool']).columns

show_heading("Column Types")

for col in X_all.columns:
    print(f"Column '{col}' dtype={X_all[col].dtype}")

print("numeric_cols:", numeric_cols)
print("cat_cols:", cat_cols)
print("bool_cols:", bool_cols)

all_cols = set()
all_cols = all_cols.union(set(numeric_cols))
all_cols = all_cols.union(set(cat_cols))
all_cols = all_cols.union(set(bool_cols))

check_cols = set(X_all.columns)

if all_cols != check_cols:
    print("ERROR! missing columns")
else:
    print("Column typing done.")

all_cols

Column Types

Column 'year' dtype=float64
Column 'make' dtype=object
Column 'model' dtype=object
Column 'condition' dtype=object
Column 'cylinders' dtype=object
Column 'fuel' dtype=object
Column 'odometer' dtype=float64
Column 'title_status' dtype=object
Column 'transmission' dtype=object
Column 'drive' dtype=object
Column 'size' dtype=object
Column 'type' dtype=object
Column 'paint_color' dtype=object
Column 'state' dtype=object
numeric_cols: Index(['year', 'odometer'], dtype='object')
cat_cols: Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')
bool_cols: Index([], dtype='object')
Column typing done.


{'condition',
 'cylinders',
 'drive',
 'fuel',
 'make',
 'model',
 'odometer',
 'paint_color',
 'size',
 'state',
 'title_status',
 'transmission',
 'type',
 'year'}

In [10]:
# We are using category_encoders package for one-hot encoding.
# https://contrib.scikit-learn.org/category_encoders/

# uncomment and run this if needed.
!pip install category_encoders

In [57]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

In [58]:
# Convert categorical columns to Pandas category type


cats = X_all.select_dtypes(exclude=np.number).columns.tolist()
print("cats:")
display(cats)
for col in cats:
    X_all[col] = X_all[col].astype('category')

display(X_all.dtypes)
display(X_all.head())

cats:


['make',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'state']

year             float64
make            category
model           category
condition       category
cylinders       category
fuel            category
odometer         float64
title_status    category
transmission    category
drive           category
size            category
type            category
paint_color     category
state           category
dtype: object

,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,az
1,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,ar
2,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,fl
3,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,ma
4,2011.406247,ford,f-150,good,6 cylinders,gas,92647.432999,clean,automatic,4wd,full-size,sedan,white,nc


## Data Dictionary

These are the columns we used from the Craigslist dataset.

|     Name     |  dtype   | Description | Null % | Notes |
|--------------|----------|--------------|--------------|--------------|
| price | float64 | Listed sale price | 0.0 |  | 
| year | float64 | Year of manufacture | 0.002823 | n/a |
| make | category | Auto manufacturer name | 0.041337 | Renamed from manufacturer |
| model | category | Car model name | 0.012362 | n/a |
| condition | category | Listed condition of vehicle | 0.407852 | n/a |
| cylinders | category | Number of engine cylinders | 0.416225 | n/a |
| fuel | category | Type of fuel used by vehicle (i.e. gas, deisel, electric) | 0.007058 | n/a |
| odometer | float64 | Odometer mileage of vehicle | 0.010307 | n/a |
| title_status | category | Status of vehicle title (i.e. clean, salvage) | 0.019308 | n/a |
| transmission | category | Type of vehicle transmission (i.e. automatic, manual) | 0.005988 | n/a |
| drive | category | Drive type of vehicle (i.e. fwd, 4wd) | 0.305863 | n/a |
| size | category | Size of vehicle(i.e. full-size) | 0.717675 | n/a |
| type | category | Vehicle type, (i.e. sedan,pickup) | 0.217527 | n/a |
| paint_color | category | Exterior paint color of vehicle | 0.305011 | n/a |
| state | category | State of vehicle location(i.e. WA, PI) | 0.000000 | n/a |

### Unused Columns

These columns we dropped after EDA before cleaning the data set.


    'id', 'url', 'region', 'VIN', 'region_url', 'county', 'lat', 'long', 'description', 'image_url', 'posting_date'
    
    
We dropped id, url, VIN, image_url because they are likely unique to each listing and uninformative.

We dropped posting_date because the dataset only has data from a limited time frame so this field is not likely to have a long enough date range to determine things like seasonality.


We dropped the columns region, region_url, county, lat, and long so that we could just use state as a single more easily usable column for the effect of the vehicle location on price.



In [60]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=193)


In [61]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error


In [24]:
# TODO: Try LeaveOneOut becuase it makes feature importance output easier.
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[
                                    ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"), cat_cols),
                                    #("leave_one_out_encoder", ce.LeaveOneOutEncoder(return_df=False), cat_cols),
                                    ('scale_data', StandardScaler(), numeric_cols)
                                ])

In [66]:
# now let's add the XGBoost version
# try to get an up to date version.
#!pip install --user --upgrade xgboost

In [67]:
import xgboost as xgb

In [68]:
# on the Mac GPUs (at least on my M2 macbook air), the CUDA libraries that XGBoost uses do not support the GPU.
# This means we are stuck with CPU for XGBoost or run it on something else.
import platform
platform.processor()
plat = platform.system()
print("Platform: ", plat)

Platform:  Darwin


In [69]:
# hyperparameters for XGBoost

# If you have GPU support, enable with this.
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
params = {}
params['objective'] = "reg:squarederror"
params['learning_rate'] = .01 # learning_rate



In [70]:
# Setup pipelines for comparison


linear_pipeline = Pipeline(steps=[
                                  ('pre_processing',pre_process),
                                  ('reg', LinearRegression())
                                 ])

ridge_pipeline = Pipeline(steps=[('pre_processing',pre_process),
                                 
                                 ('reg', SGDRegressor(penalty="l2"))
                                 ])

lasso_pipeline = Pipeline(steps=[('pre_processing',pre_process),
                                 ('reg', SGDRegressor(penalty="l1"))
                                 ])


rf_pipeline = Pipeline(steps=[('pre_processing',pre_process),
                                 ('reg', RandomForestRegressor(max_depth=10,random_state=191))
                                 ])

# NOTE: Add tree_method="gpu_hist" here as well if supported and desired.
xgb_reg = xgb.XGBRegressor(objective=params['objective'], learning_rate=params['learning_rate'])

xgb_pipeline = Pipeline(steps=[('pre_processing',pre_process),
                                 ('reg', xgb_reg)
                                 ])

pipelines = {
    'linear':linear_pipeline,
    'ridge': ridge_pipeline,
    'lasso': lasso_pipeline,
    'random_forest': rf_pipeline,
    'xgboost': xgb_pipeline
}


In [71]:
from sklearn.model_selection import cross_validate

In [72]:
def cross_validate_pipeline(label, pipeline):    
    # scoring = "neg_mean_absolute_percentage_error"
    scoring = "neg_mean_squared_error"
    n_cv_folds = 3

    print("cross validate for:", label)
    result = cross_validate(pipeline, X_all, y_all, cv=n_cv_folds, scoring=scoring)
    display(result)

    return result

In [73]:
def plot_residuals(predict_train, predict_test):

    plt.scatter(
        predict_train,
        predict_train - y_train,
        c='blue',
        marker='o',
        label='Training data')

    plt.scatter(
        predict_test,
        predict_test - y_test,
        c='lightgreen',
        marker='s',
        label='Test data')
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.legend(loc='upper left')
    plt.hlines(y=0, xmin=-15000, xmax=55000, lw=2, color='red')
    
    plt.tight_layout()
    plt.show()

In [74]:
def show_samples(X_vals, y_vals, predictions):
    samples = X_vals.copy()
    samples[target_col] = y_vals
    samples['predicted_price'] = predictions
    display(samples)


In [76]:
from sklearn.metrics import r2_score

def score_pipeline(pipeline):
    predict_train = pipeline.predict(X_train)
    predict_test = pipeline.predict(X_test)
    
    score = pipeline.score(X_test, y_test)
    r2 = r2_score(y_test, predict_test)
    # r2 == https://en.wikipedia.org/wiki/Coefficient_of_determination
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
    
    train_rmse = mean_squared_error(y_train, predict_train)**(0.5)
    test_rmse = mean_squared_error(y_test, predict_test)**(0.5)
    
    print('RMSE of train data: ', train_rmse)
    print('RMSE of test data: ',  test_rmse)
    print("model score: %.3f" % score)  # r2 anyway
    print("R2 score", r2)
    
    scores = {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'score': score,
        'r2_score': r2, 
    }
    results = {
        'scores':scores,
        'predict_train': predict_train,
        'predict_test': predict_test
    }
    print("scores:", scores)
    return results

In [94]:
import joblib
from datetime import datetime

def save_model(model_name, model, use_ts=True):
    if use_ts:
        rn = datetime.now()
        dts = '_' + rn.strftime("%Y_%m_%d_%H_%M_%S")
    else:
        dts = ''

    dest_path = f"model_{model_name}{dts}.pkl"
    print(f"Saving model to {dest_path}")
    joblib.dump(model, dest_path)
    
    return dest_path

def load_model(model_path):
    print(f"Loading model from {model_path}")
    model = joblib.load(model_path)
    return model


In [77]:
def eval_pipeline(label, pipeline):
    cross_validate_pipeline(label, pipeline)
    
    rmod = pipeline.fit(X_train,y_train)
    display(rmod)
    
    results = score_pipeline(pipeline)
     
    show_samples(X_test, y_test, results['predict_test'])
    plot_residuals(results['predict_train'], results['predict_test'])
    
    return rmod, results

Evaluating linear

cross validate for: linear


{'fit_time': array([7.69031191, 9.06415009, 8.88298392]),
 'score_time': array([0.17095971, 0.16862893, 0.17224813]),
 'test_score': array([-7.19701281e+07, -1.35943948e+08, -1.66982678e+08])}

Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')),
                                                 ('scale_data',
                                                  StandardScaler(),
                                                  Index(['year', 'odometer'], dtype='object'))])),
                ('reg', LinearRegression())])

RMSE of train data:  6456.24166019595
RMSE of test data:  7201.659534204902
model score: 0.744
R2 score 0.7436947354564545
scores: {'train_rmse': 6456.24166019595, 'test_rmse': 7201.659534204902, 'score': 0.7436947354564545, 'r2_score': 0.7436947354564545}


,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,predicted_price
373916,2017.0,ford,edge sel awd gas suv auto,good,6 cylinders,gas,28669.0,clean,automatic,4wd,full-size,SUV,white,wa,25999.0,25377.851460
379496,2018.0,mitsubishi,outlander,good,6 cylinders,gas,76935.0,clean,automatic,4wd,full-size,SUV,black,wa,18999.0,19616.974809
99114,2011.0,chevrolet,impala,fair,6 cylinders,gas,172000.0,clean,automatic,4wd,full-size,sedan,white,fl,3800.0,2979.288157
359203,2018.0,nissan,frontier,excellent,6 cylinders,gas,55020.0,clean,automatic,4wd,mid-size,pickup,red,vt,23998.0,23816.293046
105068,2012.0,hyundai,veloster,good,4 cylinders,gas,170000.0,clean,automatic,fwd,full-size,sedan,white,fl,1495.0,81.927880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245394,2011.0,kia,sorento,like new,6 cylinders,gas,132000.0,clean,automatic,4wd,full-size,SUV,white,ny,8950.0,7414.851403
137137,2015.0,gmc,terrain,good,4 cylinders,gas,90927.0,clean,automatic,fwd,full-size,SUV,grey,il,11495.0,13896.115024
75433,2008.0,ford,focus,good,4 cylinders,gas,130000.0,clean,automatic,fwd,full-size,sedan,silver,ct,3300.0,209.372351
222443,2004.0,ford,f-150,excellent,8 cylinders,gas,174000.0,clean,automatic,4wd,full-size,pickup,white,nc,10500.0,13669.965710


Evaluating ridge

cross validate for: ridge


{'fit_time': array([21.17253399, 24.72540784, 19.26449394]),
 'score_time': array([0.16930008, 0.16914797, 0.17317009]),
 'test_score': array([-76790030.1044103 , -59168482.6532187 , -71426721.96744277])}

Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')),
                                                 ('scale_data',
                                                  StandardScaler(),
                                                  Index(['year', 'odometer'], dtype='object'))])),
                ('reg', SGDRegressor())])

RMSE of train data:  7910.47536647925
RMSE of test data:  8007.000272340344
model score: 0.683
R2 score 0.6831658093120383
scores: {'train_rmse': 7910.47536647925, 'test_rmse': 8007.000272340344, 'score': 0.6831658093120383, 'r2_score': 0.6831658093120383}


,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,predicted_price
373916,2017.0,ford,edge sel awd gas suv auto,good,6 cylinders,gas,28669.0,clean,automatic,4wd,full-size,SUV,white,wa,25999.0,30440.893733
379496,2018.0,mitsubishi,outlander,good,6 cylinders,gas,76935.0,clean,automatic,4wd,full-size,SUV,black,wa,18999.0,21048.694290
99114,2011.0,chevrolet,impala,fair,6 cylinders,gas,172000.0,clean,automatic,4wd,full-size,sedan,white,fl,3800.0,2379.344059
359203,2018.0,nissan,frontier,excellent,6 cylinders,gas,55020.0,clean,automatic,4wd,mid-size,pickup,red,vt,23998.0,23449.203881
105068,2012.0,hyundai,veloster,good,4 cylinders,gas,170000.0,clean,automatic,fwd,full-size,sedan,white,fl,1495.0,-1816.511968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245394,2011.0,kia,sorento,like new,6 cylinders,gas,132000.0,clean,automatic,4wd,full-size,SUV,white,ny,8950.0,8318.903336
137137,2015.0,gmc,terrain,good,4 cylinders,gas,90927.0,clean,automatic,fwd,full-size,SUV,grey,il,11495.0,12478.419891
75433,2008.0,ford,focus,good,4 cylinders,gas,130000.0,clean,automatic,fwd,full-size,sedan,silver,ct,3300.0,-573.345652
222443,2004.0,ford,f-150,excellent,8 cylinders,gas,174000.0,clean,automatic,4wd,full-size,pickup,white,nc,10500.0,15568.943844


Evaluating lasso

cross validate for: lasso


/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'fit_time': array([47.86304307, 49.07962799, 48.30202794]),
 'score_time': array([0.16951704, 0.17466211, 0.16983604]),
 'test_score': array([-72426326.54686274, -56354721.9737378 , -66146447.55715746])}

/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')),
                                                 ('scale_data',
                                                  StandardScaler(),
                                                  Index(['year', 'odometer'], dtype='object'))])),
                ('reg', SGDRegressor(penalty='l1'))])

RMSE of train data:  7301.861525927246
RMSE of test data:  7510.265342183845
model score: 0.721
R2 score 0.7212576754739093
scores: {'train_rmse': 7301.861525927246, 'test_rmse': 7510.265342183845, 'score': 0.7212576754739093, 'r2_score': 0.7212576754739093}


,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,predicted_price
373916,2017.0,ford,edge sel awd gas suv auto,good,6 cylinders,gas,28669.0,clean,automatic,4wd,full-size,SUV,white,wa,25999.0,28863.993246
379496,2018.0,mitsubishi,outlander,good,6 cylinders,gas,76935.0,clean,automatic,4wd,full-size,SUV,black,wa,18999.0,20342.400693
99114,2011.0,chevrolet,impala,fair,6 cylinders,gas,172000.0,clean,automatic,4wd,full-size,sedan,white,fl,3800.0,2257.122518
359203,2018.0,nissan,frontier,excellent,6 cylinders,gas,55020.0,clean,automatic,4wd,mid-size,pickup,red,vt,23998.0,23561.209982
105068,2012.0,hyundai,veloster,good,4 cylinders,gas,170000.0,clean,automatic,fwd,full-size,sedan,white,fl,1495.0,-1293.593970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245394,2011.0,kia,sorento,like new,6 cylinders,gas,132000.0,clean,automatic,4wd,full-size,SUV,white,ny,8950.0,8074.022089
137137,2015.0,gmc,terrain,good,4 cylinders,gas,90927.0,clean,automatic,fwd,full-size,SUV,grey,il,11495.0,12906.416072
75433,2008.0,ford,focus,good,4 cylinders,gas,130000.0,clean,automatic,fwd,full-size,sedan,silver,ct,3300.0,-154.492120
222443,2004.0,ford,f-150,excellent,8 cylinders,gas,174000.0,clean,automatic,4wd,full-size,pickup,white,nc,10500.0,15037.885493


Evaluating random_forest

cross validate for: random_forest


{'fit_time': array([122.45037699, 124.37959599, 121.02246976]),
 'score_time': array([0.78482795, 0.77746511, 0.76297903]),
 'test_score': array([-73128326.53795086, -57013734.37640717, -68708431.26695243])}

Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')),
                                                 ('scale_data',
                                                  StandardScaler(),
                                                  Index(['year', 'odometer'], dtype='object'))])),
                ('reg', RandomForestRegressor(max_depth=10, random_state=191))])

RMSE of train data:  7716.302885302759
RMSE of test data:  7831.58202896849
model score: 0.697
R2 score 0.6968962165516847
scores: {'train_rmse': 7716.302885302759, 'test_rmse': 7831.58202896849, 'score': 0.6968962165516847, 'r2_score': 0.6968962165516847}


,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,predicted_price
373916,2017.0,ford,edge sel awd gas suv auto,good,6 cylinders,gas,28669.0,clean,automatic,4wd,full-size,SUV,white,wa,25999.0,29906.533979
379496,2018.0,mitsubishi,outlander,good,6 cylinders,gas,76935.0,clean,automatic,4wd,full-size,SUV,black,wa,18999.0,22592.897989
99114,2011.0,chevrolet,impala,fair,6 cylinders,gas,172000.0,clean,automatic,4wd,full-size,sedan,white,fl,3800.0,10174.097273
359203,2018.0,nissan,frontier,excellent,6 cylinders,gas,55020.0,clean,automatic,4wd,mid-size,pickup,red,vt,23998.0,29105.598735
105068,2012.0,hyundai,veloster,good,4 cylinders,gas,170000.0,clean,automatic,fwd,full-size,sedan,white,fl,1495.0,6127.289598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245394,2011.0,kia,sorento,like new,6 cylinders,gas,132000.0,clean,automatic,4wd,full-size,SUV,white,ny,8950.0,10967.242632
137137,2015.0,gmc,terrain,good,4 cylinders,gas,90927.0,clean,automatic,fwd,full-size,SUV,grey,il,11495.0,11740.977672
75433,2008.0,ford,focus,good,4 cylinders,gas,130000.0,clean,automatic,fwd,full-size,sedan,silver,ct,3300.0,6189.568493
222443,2004.0,ford,f-150,excellent,8 cylinders,gas,174000.0,clean,automatic,4wd,full-size,pickup,white,nc,10500.0,7244.964194


Evaluating xgboost

cross validate for: xgboost


{'fit_time': array([11.17632008, 11.48818994, 11.71546102]),
 'score_time': array([0.24688506, 0.25955296, 0.26255822]),
 'test_score': array([-1.50195819e+08, -1.25224929e+08, -1.46401177e+08])}

Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')),
                                                 ('scale_data',
                                                  StandardScaler(),
                                                  Index(['year', 'odometer'],...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

RMSE of train data:  11773.901348423493
RMSE of test data:  11814.299507065578
model score: 0.310
R2 score 0.31022363666902675
scores: {'train_rmse': 11773.901348423493, 'test_rmse': 11814.299507065578, 'score': 0.31022363666902675, 'r2_score': 0.31022363666902675}


,year,make,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,predicted_price
373916,2017.0,ford,edge sel awd gas suv auto,good,6 cylinders,gas,28669.0,clean,automatic,4wd,full-size,SUV,white,wa,25999.0,18917.787109
379496,2018.0,mitsubishi,outlander,good,6 cylinders,gas,76935.0,clean,automatic,4wd,full-size,SUV,black,wa,18999.0,17230.621094
99114,2011.0,chevrolet,impala,fair,6 cylinders,gas,172000.0,clean,automatic,4wd,full-size,sedan,white,fl,3800.0,5986.727539
359203,2018.0,nissan,frontier,excellent,6 cylinders,gas,55020.0,clean,automatic,4wd,mid-size,pickup,red,vt,23998.0,20742.460938
105068,2012.0,hyundai,veloster,good,4 cylinders,gas,170000.0,clean,automatic,fwd,full-size,sedan,white,fl,1495.0,5303.910156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245394,2011.0,kia,sorento,like new,6 cylinders,gas,132000.0,clean,automatic,4wd,full-size,SUV,white,ny,8950.0,7308.957520
137137,2015.0,gmc,terrain,good,4 cylinders,gas,90927.0,clean,automatic,fwd,full-size,SUV,grey,il,11495.0,7139.934082
75433,2008.0,ford,focus,good,4 cylinders,gas,130000.0,clean,automatic,fwd,full-size,sedan,silver,ct,3300.0,3903.598389
222443,2004.0,ford,f-150,excellent,8 cylinders,gas,174000.0,clean,automatic,4wd,full-size,pickup,white,nc,10500.0,4536.090332


In [95]:
all_results = []
for label, pipeline in pipelines.items():
    show_heading(f"Evaluating {label}", 2)
    rmod, results = eval_pipeline(label, pipeline)
    scores = {'label': label}
    scores.update(results['scores'])
    all_results.append(scores)

In [96]:
df = pd.DataFrame.from_dict(all_results)

show_heading("Pipeline Score Summary")
display(df)

Pipeline Score Summary

,label,train_rmse,test_rmse,score,r2_score
0,linear,6456.241660,7201.659534,0.743695,0.743695
1,ridge,7910.475366,8007.000272,0.683166,0.683166
2,lasso,7301.861526,7510.265342,0.721258,0.721258
3,random_forest,7716.302885,7831.582029,0.696896,0.696896
4,xgboost,11773.901348,11814.299507,0.310224,0.310224


In [97]:
def plot_evals_result(results):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    
    x = None
    
    lines = {}
    for key, values in results.items():
        print("key:", key)
        if x is None:
            n_rounds = len(values)
            x = np.linspace(0, n_rounds, n_rounds)
            
        lines[key],  = ax.plot(x, values, label=key)
        ax.legend()

    plt.show()
    

In [98]:
# Callback for xgboost to save error metrics by epoch.
class EvalCapture(xgb.callback.TrainingCallback):
    
    def __init__(self, rounds):
        self.rounds = rounds
        self.lines = {}
        self.x = np.linspace(0, self.rounds, self.rounds)
        self.evals_log = None
        
    def _get_key(self, data, metric):
        return f'{data}-{metric}'

    def evals_result(self):
        # return results as dict of array of values for each round.
        results = {}
        
        #display(self.evals_log) # TODO:
        for data, metric in self.evals_log.items():
            for metric_name, log in metric.items():
                key = self._get_key(data, metric_name)
                results[key] = log
        return results
         
        
    def after_iteration(self, model, epoch, evals_log):
        # Keep most recent log
        self.evals_log = evals_log
        return False

In [99]:
def xgboost_fit(X_train, y_train, X_test, y_test, enable_cat, n_rounds = 5000):
    # This is the non pipeline version for helping with measuring feature importance.

    # Convert to the format XGBoost lib expects.
    dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=enable_cat)
    dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=enable_cat)

    evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]

    evals_capture = EvalCapture(n_rounds)
    
    model = xgb.train(
      params=params,
      dtrain=dtrain_reg,
      num_boost_round=n_rounds,
      evals=evals,
      verbose_eval=50, # print every N rounds
      early_stopping_rounds=10,  # stop if no improvement to validation loss in N rounds.
      callbacks=[evals_capture]
    )

    print("results")
    evals_result = evals_capture.evals_result()
    plot_evals_result(evals_result)
       
    
    predict_test = model.predict(dtest_reg)
    predict_train = model.predict(dtrain_reg)
    
    print('RMSE of train data: ', mean_squared_error(y_train, predict_train)**(0.5))
    print('RMSE of test data: ',  mean_squared_error(y_test, predict_test)**(0.5))

    r2 = r2_score(y_test, predict_test)
    print('R2 Score of test data:', r2)
        
    return model
# xgboost_fit() 

[0]	validation-rmse:23276.21083	train-rmse:23245.86123
[50]	validation-rmse:15825.14740	train-rmse:15787.15004
[100]	validation-rmse:11846.13175	train-rmse:11804.79298
[150]	validation-rmse:9807.73945	train-rmse:9764.38696
[200]	validation-rmse:8811.68471	train-rmse:8767.24334
[250]	validation-rmse:8291.68103	train-rmse:8248.36946
[300]	validation-rmse:8002.62976	train-rmse:7959.64331
[350]	validation-rmse:7813.77147	train-rmse:7768.67821
[400]	validation-rmse:7686.77372	train-rmse:7638.01660
[450]	validation-rmse:7564.41037	train-rmse:7512.56543
[500]	validation-rmse:7457.91076	train-rmse:7401.50686
[550]	validation-rmse:7368.23549	train-rmse:7307.31746
[600]	validation-rmse:7298.37426	train-rmse:7233.98581
[650]	validation-rmse:7235.33614	train-rmse:7167.78743
[700]	validation-rmse:7181.89733	train-rmse:7111.66956
[750]	validation-rmse:7134.16874	train-rmse:7061.67814
[800]	validation-rmse:7090.25170	train-rmse:7014.23279
[850]	validation-rmse:7050.89341	train-rmse:6969.90444
[900]	v

RMSE of train data:  5424.530306675746
RMSE of test data:  5742.401245939235
R2 Score of test data: 0.837040596773232


In [100]:
# Train a separate model to do feature importance
params['learning_rate'] = .01 # learning_rate
model = xgboost_fit(X_train, y_train, X_test, y_test, enable_cat=True) 


<AxesSubplot: title={'center': 'Feature importance'}, xlabel='F score', ylabel='Features'>

In [101]:
n_features_max = 50

from xgboost import plot_importance
plot_importance(model, max_num_features=20)

In [102]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
    
n_pca_comp = 10

# Using LeaveOneOut and PCA
cat_encoder = Pipeline([
    ('LOO', ce.LeaveOneOutEncoder(return_df=False)),
    ('PCA', PCA(n_components = n_pca_comp,random_state=193))
])

alt_pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[
                                    #("leave_one_out_encoder", ce.LeaveOneOutEncoder(return_df=False), cat_cols),
                                    ("cat_encoder", cat_encoder, cat_cols),
                                    ('scale_data', StandardScaler(), numeric_cols),
                                ])

In [103]:
def train_random_forest(xreg_train, yreg_train, xreg_test, yreg_test):


    RF_regressor = Pipeline([('pre_processing',alt_pre_process),
                             ('reg', RandomForestRegressor(max_depth=15,random_state=191))])

    RF_regressor.fit(xreg_train, yreg_train)

    predict_train = RF_regressor.predict(xreg_train)
    predict_test = RF_regressor.predict(xreg_test)

    train_rmse = mean_squared_error(yreg_train, predict_train)**(0.5)
    test_rmse = mean_squared_error(yreg_test, predict_test)**(0.5)
  
    score = RF_regressor.score(X_test, y_test)
    r2 = r2_score(y_test, predict_test)
    
    train_rmse = mean_squared_error(y_train, predict_train)**(0.5)
    test_rmse = mean_squared_error(y_test, predict_test)**(0.5)
    
    print('RMSE of train data: ', train_rmse)
    print('RMSE of test data: ',  test_rmse)
    print("model score: %.3f" % score)
    print("R2 score", r2)
    
    
    return RF_regressor


In [104]:
print("Random Forest")
train_random_forest(X_train, y_train, X_test, y_test)    


Random Forest
RMSE of train data:  4810.785364020331
RMSE of test data:  5715.183210629658
model score: 0.839
R2 score 0.8385817371382678


Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_encoder',
                                                  Pipeline(steps=[('LOO',
                                                                   LeaveOneOutEncoder(return_df=False)),
                                                                  ('PCA',
                                                                   PCA(n_components=10,
                                                                       random_state=193))]),
                                                  Index(['make', 'model', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')),
                                                 ('scale_data',
                                                  StandardScaler(),
                                                  Index(['year', 'odometer'], dtype='object'))])),
                ('reg', RandomForestRegressor(max_depth=15, random_state=191))])

In [105]:
def score_xgb_model(X_train, y_train, X_test, y_test):
    n_rounds = 5000    
    xgb_model = xgboost_fit(X_train, y_train, X_test, y_test, enable_cat=False, n_rounds=n_rounds) 
    return xgb_model

XGBoost
[0]	validation-rmse:21393.89691	train-rmse:21347.82891
[50]	validation-rmse:6575.08259	train-rmse:6265.55465
[100]	validation-rmse:6371.88206	train-rmse:6029.40295
[150]	validation-rmse:6261.56595	train-rmse:5881.05271
[200]	validation-rmse:6181.25838	train-rmse:5762.23117
[250]	validation-rmse:6111.73536	train-rmse:5655.58060
[300]	validation-rmse:6052.78804	train-rmse:5565.38557
[350]	validation-rmse:6015.75131	train-rmse:5500.14028
[400]	validation-rmse:5962.05489	train-rmse:5413.85683
[450]	validation-rmse:5917.03095	train-rmse:5336.37495
[500]	validation-rmse:5880.93249	train-rmse:5275.08400
[550]	validation-rmse:5850.75523	train-rmse:5218.23785
[600]	validation-rmse:5817.25672	train-rmse:5155.51112
[650]	validation-rmse:5788.28259	train-rmse:5101.94557
[700]	validation-rmse:5764.02678	train-rmse:5054.45397
[750]	validation-rmse:5735.79946	train-rmse:5001.30723
[800]	validation-rmse:5715.66124	train-rmse:4956.72658
[850]	validation-rmse:5694.98977	train-rmse:4912.04233
[90

RMSE of train data:  3148.6374127464414
RMSE of test data:  5077.025190920731
R2 Score of test data: 0.872617141105975


In [106]:
print("XGBoost")

alt_pre_process.fit(X_train, y_train)

X_pre_train =  alt_pre_process.transform(X_train)
X_pre_test =  alt_pre_process.transform(X_test)

# TODO: set learning rate higher and rerun
params['learning_rate'] = .1 # learning_rate
xgb_model = score_xgb_model(X_pre_train, y_train, X_pre_test, y_test)

In [107]:
# save the model
save_model("xgboost", model, use_ts=True)

Saving model to model_xgboost_2023_03_12_19_26_53.pkl


'model_xgboost_2023_03_12_19_26_53.pkl'